<a href="https://colab.research.google.com/github/MRGotIdeas/twitter_fakenews/blob/main/00_get_fakenews_websites.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Récupération des sites de fake news

LeMonde met à disposition un fichier json contenant des sites d'information ainsi que leur fiabilité. Ce notebook vise à récupérer cette information, et l'enregistrer au format csv. 

Voir issue #x

#### Import packages

In [ ]:
import pandas as pd
import json
import re

In [ ]:
path_data = '/content/drive/MyDrive/Colab Notebooks/PFE/twitter_fakenews/data/'

TEST github

## Import du fichier json

NB : l'enregistrer à partir du site leMonde au format "données brutes" et non au format json

In [ ]:
with open(f'{path_data}/fakenews_websites_lemonde.json', 'r') as json_file:
  data = json.loads(json_file.read())

In [ ]:
data.keys()

dict_keys(['sites', 'urls'])

**Good to know** : 
- La clé `sites` représente certaines informations comme la fiabilité, un descriptif du site, le nom du site 
- La clé `urls` donne l'ensemble des liens qui appartiennent au site : lien facebook, twitter, youtube, etc

In [ ]:
# un exemple de site 
data['sites']['1393']

In [ ]:
# on cherche l'ensemble des sites qui ont l'identifiant 1393 : buzzarena
[key for key, value in data['urls'].items() if value == 1393]

In [ ]:
# on rajoute la contrainte : on ne veut que les comptes twitter
[key for key, value in data['urls'].items() if value == 1393 and re.search("twitter", key)]

## Formatage des données

On va faire matcher les comptes twitter avec les infos sur les sites

In [ ]:
def format_df_fakenews(json_file) :

  df_sites = pd.DataFrame.from_dict(json_file["sites"], 
                                    orient="index", 
                                    columns=["liability", "description","name","namelow"])\
                                    .reset_index()\
                                    .rename(columns={'index': 'unique_id'})
  
  df_url = pd.DataFrame.from_dict(json_file["urls"], orient="index").reset_index()
  df_url.columns = ["path_website", "unique_id"]

  return df_sites


In [ ]:
df_test = pd.DataFrame.from_dict(data['sites'], 
                                 orient="index", 
                                 columns=["liability", "description","name","namelow"])\
                                 .reset_index()\
                                 .rename(columns={'index': 'unique_id'})
df_test.shape

(1343, 5)

In [ ]:
df_test.unique_id.describe()

count     1343
unique    1343
top       1250
freq         1
Name: unique_id, dtype: object

In [ ]:
def label_liability(df) : 
  df['liability_label'] = df['liability'].map({1:'site parodique', 
                                               2:'site qui diffuse régulièrement de la fausse information',
                                               3:'site à la fiabilité douteuse',
                                               4:'site réputé fiable'})
  
  return df

df_test = label_liability(df_test)

In [ ]:
df_test.head(3)

In [ ]:
df_test['liability_label'].value_counts()

site qui diffuse régulièrement de la fausse information    665
site réputé fiable                                         311
site à la fiabilité douteuse                               243
site parodique                                             124
Name: liability_label, dtype: int64

In [ ]:
df_url = pd.DataFrame.from_dict(data["urls"], orient="index").reset_index()
df_url.columns = ["path_website", "unique_id"]

In [ ]:
df_url.head(3)

,path_website,unique_id
0,francesoir.fr,1982
1,facebook.com/FranceSoir,1982
2,twitter.com/france_soir,1982


In [ ]:
count_sites_by_id = df_url.groupby('unique_id').size().reset_index()
count_sites_by_id.columns = ['unique_id','count']
count_sites_by_id['count'].value_counts()

1    710
3    313
2    230
4     80
Name: count, dtype: int64

In [ ]:
df_url['facebook'] = df_url.path_website.apply(lambda txt : len(re.findall("[fF]acebook.+", txt))>0)
df_url['youtube'] = df_url.path_website.apply(lambda txt : len(re.findall("[yY]outube.+", txt))>0)
df_url['twitter'] = df_url.path_website.apply(lambda txt : len(re.findall("twitter.+", txt))>0)

In [ ]:
df_url.head()

,path_website,unique_id,facebook,youtube,twitter
0,francesoir.fr,1982,False,False,False
1,facebook.com/FranceSoir,1982,True,False,False
2,twitter.com/france_soir,1982,False,False,True
3,regenere.org,1981,False,False,False
4,facebook.com/thierry.rgnr,1981,True,False,False


In [ ]:
test = df_url.groupby("unique_id").nunique().reset_index()
test.head(3)

,unique_id,path_website,facebook,youtube,twitter
0,573,4,2,2,2
1,574,3,2,1,2
2,575,3,2,1,2


In [ ]:
len(test.loc[test['twitter']==2])

414